# クローリングとスクレイピング

## データのダウンロード(urllib)<br>
<strong>urllib : URLを扱うモジュールを集めたパッケージ</strong>

### ファイルの保存方法

In [1]:
#ライブラリの読み込み
import urllib.request

#URLと保存パスを指定
url = "http://uta.pw/shodou/img/28/214.png"
savename = "test.png"

#ダウンロード
urllib.request.urlretrieve(url, savename)
print("保存しました")

保存しました


In [4]:
#pythonのメモリ上に取得する
mem = urllib.request.urlopen(url).read()

#ファイルへ保存
with open(savename, mode ="wb") as f:
    f.write(mem)
    print("保存しました")

保存しました


### Webからデータを取得

In [6]:
#IP確認アクセスして結果を表示する
import urllib.request

#データを取得する
url = "http://api.aoikujira.com/ip/ini"
res = urllib.request.urlopen(url)
data = res.read()

#バイナリを文字列に変換
text = data.decode("utf-8")
print(text)

[ip]
API_URI=http://api.aoikujira.com/ip/get.php
REMOTE_ADDR=153.156.85.18
REMOTE_HOST=p6762018-ipngnfx01marunouchi.tokyo.ocn.ne.jp
REMOTE_PORT=47970
HTTP_HOST=api.aoikujira.com
HTTP_USER_AGENT=Python-urllib/3.6
HTTP_ACCEPT_LANGUAGE=
HTTP_ACCEPT_CHARSET=
SERVER_PORT=80
FORMAT=ini




### URLにパラメータをつけてリクエストを送信する方法<br>
fmt : データのフォーマットをjsonからxmlで指定<br>
zn : 住所を知りたい郵便番号を指定

In [10]:
import urllib.request
import urllib.parse

API = "http://api.aoikujira.com/zip/xml/get.php"

#パラメーターをURLエンコードする
values = {
    'fmt' : 'xml',
    'zn' : '1500042'
}

params = urllib.parse.urlencode(values)

#リクエスト用のURLを生成
url = API + "?" + params
print("url = ", url)

#ダウンロード
data = urllib.request.urlopen(url).read()
text = data.decode("utf-8")
print(text)

url =  http://api.aoikujira.com/zip/xml/get.php?fmt=xml&zn=1500042
<?xml version="1.0" encoding="utf-8" ?>
<address result="1">
<header>
  <result>1</result>
  <api>api.aoikujira.com/zip</api>
  <version>1.1</version>
</header>
<value>
  <zip>1500042</zip>
  <ken>東京都</ken>
  <shi>渋谷区</shi>
  <cho>宇田川町</cho>
  <disp>東京都渋谷区宇田川町</disp>
  <kenkana>トウキョウト</kenkana>
  <shikana>シブヤク</shikana>
  <chokana>ウダガワチョウ</chokana>
</value>
</address>


### CSSセレクタを使う

In [12]:
from bs4 import BeautifulSoup 

# 解析対象となるHTML --- (※1)
html = """
<html><body>
<div id="meigen">
  <h1>トルストイの名言</h1>
  <ul class="items">
    <li>汝の心に教えよ、心に学ぶな</li>
    <li>謙虚な人は誰からも好かれる。</li>
    <li>強い人々は、いつも気取らない。</li>
  </ul>
</div>
</body></html>
"""

# HTMLを解析する --- (※2)
soup = BeautifulSoup(html, 'html.parser')

# 必要な部分をCSSクエリで取り出す
# | タイトル部分を取得 --- (※3)
h1 = soup.select_one("div#meigen > h1").string
print("h1 =", h1)

# | リスト部分を取得 --- (※4)
li_list = soup.select("div#meigen > ul.items > li")
for li in li_list:
	print("li =", li.string)

h1 = トルストイの名言
li = 汝の心に教えよ、心に学ぶな
li = 謙虚な人は誰からも好かれる。
li = 強い人々は、いつも気取らない。


## リンク先を丸ごとダウンロード

### 相対パスを展開

In [13]:
from urllib.parse import urljoin

base = "http://example.com/html/a.html"

print( urljoin(base, "b.html"))

http://example.com/html/b.html


## 実際にダウンロードする例<br>
あんまりちゃんと理解してない

In [14]:
# Pythonのマニュアルを再帰的にダウンロード 
# モジュールの取り込み --- (※1)
from bs4 import BeautifulSoup
##インターネット上のデータを取得
from urllib.request import *
##URLの解決を行う
from urllib.parse import *
##ディレクトリ作成
from os import makedirs
##パス、スリープ、正規表現
import os.path, time, re

# HTMLファイルの解析を行ったかどうか判断変数 --- (※2)
# →重複して解析をしないため
proc_files = {}

# HTML内にあるリンクを抽出する関数 --- (※3)
def enum_links(html, base):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("link[rel='stylesheet']") # CSS
    links += soup.select("a[href]") # リンク
    result = []
    # href属性を取り出し、リンクを絶対パスに変換 --- (※4)
    for a in links:
        href = a.attrs['href']
        url = urljoin(base, href)
        result.append(url)
    return result

# ファイルをダウンロードし保存する関数 --- (※5)
def download_file(url):
    o = urlparse(url)
    savepath = "./" + o.netloc + o.path
    if re.search(r"/$", savepath): # ディレクトリならindex.html
        savepath += "index.html"
    savedir = os.path.dirname(savepath)
    # 既にダウンロード済み?
    if os.path.exists(savepath): return savepath
    # ダウンロード先のディレクトリを作成
    if not os.path.exists(savedir):
        print("mkdir=", savedir)
        makedirs(savedir)
    # ファイルをダウンロード --- (※6)
    try:
        print("download=", url)
        urlretrieve(url, savepath)
        time.sleep(1) # 礼儀として1秒スリープ --- (※7)
        return savepath
    except:
        print("ダウンロード失敗:", url)
        return None        

# HTMLを解析してダウンロードする関数 --- (※8)
def analize_html(url, root_url):
    savepath = download_file(url)
    if savepath is None: return
    if savepath in proc_files: return # 解析済みなら処理しない --- (※9)
    proc_files[savepath] = True
    print("analize_html=", url)
    # リンクを抽出 --- (※10)
    html = open(savepath, "r", encoding="utf-8").read()
    links = enum_links(html, url)
    for link_url in links:
        # リンクがルート以外のパスを指していたら無視 --- (※11)
        if link_url.find(root_url) != 0:
            if not re.search(r".css$", link_url): continue
        # HTMLか？
        if re.search(r".(html|htm)$", link_url):
            # 再帰的にHTMLファイルを解析
            analize_html(link_url, root_url)
            continue
        # それ以外のファイル
        download_file(link_url)

if __name__ == "__main__":
    # URLを丸ごとダウンロード --- (※13)
    url = "http://docs.python.jp/3.5/library/"
    analize_html(url, url)

mkdir= ./docs.python.jp/3.5/library
download= http://docs.python.jp/3.5/library/
analize_html= http://docs.python.jp/3.5/library/
mkdir= ./docs.python.jp/3.5/_static
download= http://docs.python.jp/3.5/_static/pydoctheme.css
download= http://docs.python.jp/3.5/_static/pygments.css
download= http://docs.python.jp/3.5/library/intro.html
analize_html= http://docs.python.jp/3.5/library/intro.html
download= http://docs.python.jp/3.5/library/functions.html
analize_html= http://docs.python.jp/3.5/library/functions.html


KeyboardInterrupt: 

# 高度なスクレイピング

## 会員制Webサイトにあるログインの仕組み

pythonのライブラリ[requests]の利用

## ブラウザーを経由したスクローリング

Seleniumの利用とコマンドラインから使えるPhantomJS

## WebAPIからのデータ取得

## cronと定期的なクローリング<br>
定期的に処理を実行するためのパッケージ

In [15]:
import urllib.request as request
import datetime
import json

# Web APIにアクセス
API = "http://api.aoikujira.com/kawase/get.php?code=USD&format=json"
json_str = request.urlopen(API).read().decode("utf-8")
data = json.loads(json_str)
print("1USD="+data["JPY"]+"JPY")

# 保存ファイル名を決定
t = datetime.date.today()
fname = t.strftime("%Y-%m-%d") + ".json"
with open(fname, "w", encoding="utf-8") as f:
    f.write(json_str)

1USD=110.45904JPY


【使い方】
①crontabの起動<br>
＄crontab -e <br>
②動作を記述<br>
07***/path/to/python3 /path/to/everyday-kawase.py
７時に上記のコード（everyday-kawase.py)を毎日実行

# データベース

　あんまり詳しく書かれていないのでパス